In [31]:
import string
import re
from nltk.tokenize import TweetTokenizer, word_tokenize 

In [13]:
import docx
from docx import Document
dolch_story = docx.Document('C:\\Users\\Shreya Singh\\Downloads\\dolch_story.docx')
paras = []
for paragraph in dolch_story.paragraphs:
    paras.append(paragraph.text.lower())

In [5]:
paras

['(This passage contains all of the 220 Dolch Basic Sight Words.)',
 '',
 'The Best Thing In The World',
 'Once upon a time, there were four brothers who lived in a far away land. Their father was an old king. One day he said, "I will not live long now. Today you must start out into the world. In a year, bring back the best thing you have found. The one who can pick the best thing shall be the new king." The first brother said, "I will look in every city or town. I will buy the best thing I can for my father." The next two brothers said, "We will both go on fast ships over the sea. We will find something better." The last brother said, "I am going to ask the people here in our own land to tell me the best thing." The other three brothers began to laugh. "Then you will never be king!" They said. ',
 '',
 'The last brother started off. When he had gone about six miles, he met a man. "What do you carry in those big bags?" he asked. "The best thing in the world," said the man. "These are f

In [43]:
def clean_para(str1):
    para=re.sub(r' ., +, *, ?, ^, $, (, ), [, ], {, }, |, \.,"''", ','',str(str1))
    para=re.sub(r'\n','',str(para))
    para=re.sub(r'\'','',str(para))
    para=re.sub(r',','',str(para))
    tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    tweet_tokens=tokenizer.tokenize(para)
    word_clean=[]
    for words in tweet_tokens:
        if words not in string.punctuation:
            words=words.lower()
            word_clean.append(words)
            temp=set(word_clean)
    return temp

In [7]:
tok_words=clean_para(paras)

In [8]:
def match_pattern_words(words, pattern,sen):
    matched_word = []
    for word in words:
        if(re.match(pattern,word)):
            matched_word.append(word)
    pos=[]
    for x in matched_word:
        matches = re.finditer(x, sen)
        matches_positions = [match.start() for match in matches]
        pos.append(matches_positions)
    return matched_word, pos

In [9]:
import time

In [15]:
start_time = time.time()
match_pattern_words(tok_words, '.*ing',str(paras))
print(" %s seconds" % (time.time() - start_time))

 0.0010013580322265625 seconds


In [16]:
match_pattern_words(tok_words, '.*ing',str(paras))

(['something',
  'bring',
  'king',
  'sing',
  'thing',
  'things',
  'wing',
  'going',
  'nothing',
  'crying'],
 [[599],
  [303, 3007, 3098, 3262, 3298],
  [201, 398, 785, 2524, 2572, 2962, 3115, 3213, 3277, 3431],
  [2182],
  [82, 323, 375, 485, 603, 712, 948, 2137, 2346, 2851, 3072, 3197, 3383],
  [3072],
  [1662],
  [647],
  [3195],
  [1310, 1351]])

In [20]:
import gensim 
from gensim.models import Word2Vec 

In [54]:
def clean_para_sen(sentence):
    sentence=re.sub(r' ., +, *, ?, ^, $, (, ), [, ], {, }, |, \.,"''", ','',str(sentence))
    sentence=re.sub(r'\n','',str(sentence))
    sentence=re.sub(r'\'','',str(sentence))
    sentence=re.sub(r',','',str(sentence))
    sentence=re.sub(r'``','',str(sentence))
    data = []
    #Sentokenizer=sent_tokenize(preserve_case=False,strip_handles=True,reduce_len=True)
    for i in sent_tokenize(sentence): 
        temp1=[]
    # tokenize the sentence into words 
        for j in word_tokenize(i):
            if j not in string.punctuation:
                temp1.append(j.lower()) 
  
        data.append(temp1)
    return data

In [56]:
clean_words_matrix=clean_para_sen((paras))

In [57]:
clean_words_matrix[2]

['their', 'father', 'was', 'an', 'old', 'king']

In [64]:
import warnings

In [78]:
def cont_bag_of_words(vect,word1, word2):
    warnings.filterwarnings(action = 'ignore')
    model1 = gensim.models.Word2Vec(vect, min_count = 1, size = 1000, window = 5) 
    return model1.similarity(word1, word2)    

In [79]:
cont_bag_of_words(clean_words_matrix,'father', 'king')

0.033335455

In [80]:
cont_bag_of_words(clean_words_matrix,'crying', 'cried')

0.01934857

In [82]:
def skip_gram(vect,word1, word2):
    warnings.filterwarnings(action = 'ignore')
    model2 = gensim.models.Word2Vec(vect, min_count = 1, size = 1000, window = 5, sg = 1) 
    return model2.similarity(word1, word2)  

In [84]:
skip_gram(clean_words_matrix,'crying', 'cried')

0.023282867

In [114]:
def skip_gram_similarity(word1): #we can use this to find out if a word is present in the article or not
    temp1=[]
    temp=[]
    warnings.filterwarnings(action = 'ignore')
    for x in clean_words_matrix:
        for y in x:
            temp1.append(y)
    temp1=set(temp1)
    for word in temp1:
        model2 = gensim.models.Word2Vec(clean_words_matrix, min_count = 1, size = 1000, window = 5, sg = 1)
        if model2.similarity(word1, word) >= .080:
            temp.append(word)
    return temp

In [115]:
skip_gram_similarity('father')

['was', 'big', 'gone', 'day', 'is', 'in', 'father', 'miles']

In [116]:
skip_gram_similarity('cried')

['last', 'take', 'cried', 'would', 'sea']

In [125]:
def find_pos_of_similar_words(str1,sen):
    pos=[]
    for x in skip_gram_similarity(str1):
        print(x)
        matches = re.finditer(x, sen)
        matches_positions = [match.start() for match in matches]
        pos.append(matches_positions)
    return pos

In [126]:
find_pos_of_similar_words('cried',str(paras))

last
take
cried
would
sea


[[622, 815, 3127, 3234, 3350],
 [1818, 2860],
 [3391],
 [1073, 1728, 2071, 2493, 2540, 2625, 2657, 2715],
 [581]]